# Tratamento do dataset df1

### Processo de carregamento dos dados

In [1]:
# Instalação das bibliotecas
!pip install pyspark

In [2]:
# Importações
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, regexp_extract, col, count, sum, when, countDistinct
from pyspark.sql.types import IntegerType, FloatType
from functools import reduce

In [3]:
# Criação do ambiente Spark
spark = SparkSession.builder \
.appName('Prata1') \
.getOrCreate()

In [4]:
# Carregamento do dataset inicial (df1)
df = spark.read.csv(r"C:\Users\fred\Documents\CursoFullStack\Projeto\Web Scraping\Dados\df1.csv", sep = ";", header = True)
df.show()

+--------------------+--------------------+--------------------+------+---------+-----------------+------------------+--------------------+
|           Categoria|             Produto|               Marca| Preco|Avaliacao|Quant. Avaliacoes|Quant. Comentários|                 URL|
+--------------------+--------------------+--------------------+------+---------+-----------------+------------------+--------------------+
|Tapete Higienico ...|Tapete Higiênico ...|               Petix| 74.49|      4.8|           2210.0|   373 comentários|https://www.merca...|
|   Racao Golden 15kg|Golden formula pr...|              Golden|191.28|      4.8|            600.0|   106 comentários|https://www.merca...|
|              Keldog|Bifinho para Cães...|              Keldog|  25.7|      4.9|            643.0|    96 comentários|https://www.merca...|
|Shampoo Cloresten...|1 Cloresten Shamp...|  Agener União Noxxi| 209.9|      4.9|            184.0|    46 comentários|https://www.merca...|
|               Cama

### Padronização de variáveis e valores

In [5]:
# Renomeação de colunas
df_coluna_ren = df.withColumnRenamed("Quant. Avaliacoes", "Quant_Avaliacoes").withColumnRenamed("Quant. Comentários", "Quant_Comentarios")
df_coluna_ren.show()

+--------------------+--------------------+--------------------+------+---------+----------------+-----------------+--------------------+
|           Categoria|             Produto|               Marca| Preco|Avaliacao|Quant_Avaliacoes|Quant_Comentarios|                 URL|
+--------------------+--------------------+--------------------+------+---------+----------------+-----------------+--------------------+
|Tapete Higienico ...|Tapete Higiênico ...|               Petix| 74.49|      4.8|          2210.0|  373 comentários|https://www.merca...|
|   Racao Golden 15kg|Golden formula pr...|              Golden|191.28|      4.8|           600.0|  106 comentários|https://www.merca...|
|              Keldog|Bifinho para Cães...|              Keldog|  25.7|      4.9|           643.0|   96 comentários|https://www.merca...|
|Shampoo Cloresten...|1 Cloresten Shamp...|  Agener União Noxxi| 209.9|      4.9|           184.0|   46 comentários|https://www.merca...|
|               Camas|Cama Pet Cac

In [6]:
# Retirada das vírgulas que marcavam milhar nos campos da coluna "Quant_Comentarios"
df_Quan_comen_tratado = df_coluna_ren.withColumn("Quant_Comentarios", regexp_replace("Quant_Comentarios", ",",""))

In [7]:
# Retirada de caracteres não numéricos da coluna "Quant_Comentarios"
df_Quan_comen_tratado = df_coluna_ren.withColumn("Quant_Comentarios", regexp_extract("Quant_Comentarios", r"(\d+)",1))
df_Quan_comen_tratado.show()

+--------------------+--------------------+--------------------+------+---------+----------------+-----------------+--------------------+
|           Categoria|             Produto|               Marca| Preco|Avaliacao|Quant_Avaliacoes|Quant_Comentarios|                 URL|
+--------------------+--------------------+--------------------+------+---------+----------------+-----------------+--------------------+
|Tapete Higienico ...|Tapete Higiênico ...|               Petix| 74.49|      4.8|          2210.0|              373|https://www.merca...|
|   Racao Golden 15kg|Golden formula pr...|              Golden|191.28|      4.8|           600.0|              106|https://www.merca...|
|              Keldog|Bifinho para Cães...|              Keldog|  25.7|      4.9|           643.0|               96|https://www.merca...|
|Shampoo Cloresten...|1 Cloresten Shamp...|  Agener União Noxxi| 209.9|      4.9|           184.0|               46|https://www.merca...|
|               Camas|Cama Pet Cac

In [8]:
# Avaliação do tipo de dados por coluna
df_Quan_comen_tratado.printSchema()

root
 |-- Categoria: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Marca: string (nullable = true)
 |-- Preco: string (nullable = true)
 |-- Avaliacao: string (nullable = true)
 |-- Quant_Avaliacoes: string (nullable = true)
 |-- Quant_Comentarios: string (nullable = true)
 |-- URL: string (nullable = true)



In [9]:
# Alteração do tipo de dados das colunas com valores numéricos
df_tipos_dados = df_Quan_comen_tratado \
    .withColumn("Preco", df_Quan_comen_tratado["Preco"].cast(FloatType())) \
    .withColumn("Quant_Avaliacoes", df_Quan_comen_tratado["Quant_Avaliacoes"].cast(IntegerType())) \
    .withColumn("Avaliacao", df_Quan_comen_tratado["Avaliacao"].cast(FloatType())) \
    .withColumn("Quant_Comentarios", df_Quan_comen_tratado["Quant_Comentarios"].cast(IntegerType()))


In [10]:
# Resultado das alterações do tipo de dados por Coluna
df_tipos_dados.printSchema()

root
 |-- Categoria: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Marca: string (nullable = true)
 |-- Preco: float (nullable = true)
 |-- Avaliacao: float (nullable = true)
 |-- Quant_Avaliacoes: integer (nullable = true)
 |-- Quant_Comentarios: integer (nullable = true)
 |-- URL: string (nullable = true)



### Tratamento de campos nulos

In [11]:
# Rastreamento dos campos nulos
df_tipos_dados.filter(reduce(lambda x, y: x | y, [col(c).isNull() for c in df_tipos_dados.columns])).show()

+--------------------+--------------------+--------------------+------+---------+----------------+-----------------+--------------------+
|           Categoria|             Produto|               Marca| Preco|Avaliacao|Quant_Avaliacoes|Quant_Comentarios|                 URL|
+--------------------+--------------------+--------------------+------+---------+----------------+-----------------+--------------------+
|    Tapete Higienico|Tapetes Higiênico...|                NULL| 47.39|      3.9|               9|                3|https://click1.me...|
| Biscoitos para Cães|Combo 4 Lata Pedi...|Ração úmida para ...| 49.42|      5.0|               2|             NULL|https://click1.me...|
|               Camas|Caminha Pet Suspe...|            Animalli|199.89|     NULL|            NULL|             NULL|https://produto.m...|
|    Tapete Higienico|3 Tapetes Higiêni...|               Petix| 298.9|      5.0|               3|             NULL|https://produto.m...|
|            Frisbees|Brinquedo Fr

In [12]:
# Totais não nulos por coluna
totais = df_tipos_dados.select([count(col(c)).alias(c) for c in df_tipos_dados.columns])
totais.show()

+---------+-------+-----+-----+---------+----------------+-----------------+----+
|Categoria|Produto|Marca|Preco|Avaliacao|Quant_Avaliacoes|Quant_Comentarios| URL|
+---------+-------+-----+-----+---------+----------------+-----------------+----+
|     2280|   2280| 2270| 2266|     1954|            1954|             1624|2280|
+---------+-------+-----+-----+---------+----------------+-----------------+----+



In [13]:
#Contagem de nulos por coluna
nulos_por_coluna = df_tipos_dados.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_tipos_dados.columns])
nulos_por_coluna.show()

+---------+-------+-----+-----+---------+----------------+-----------------+---+
|Categoria|Produto|Marca|Preco|Avaliacao|Quant_Avaliacoes|Quant_Comentarios|URL|
+---------+-------+-----+-----+---------+----------------+-----------------+---+
|        0|      0|   10|   14|      326|             326|              656|  0|
+---------+-------+-----+-----+---------+----------------+-----------------+---+



In [14]:
#Alterando valores nulos das colunas "Quant_Avaliacoes" e "Quant_Comentarios" para 0.
tratamento_nulos = df_tipos_dados.withColumn("Quant_Avaliacoes", when(col("Quant_Avaliacoes").isNull(), 0).otherwise(col("Quant_Avaliacoes"))). \
withColumn("Quant_Comentarios", when(col("Quant_Comentarios").isNull(), 0).otherwise(col("Quant_Comentarios")))
tratamento_nulos.show()

+--------------------+--------------------+--------------------+------+---------+----------------+-----------------+--------------------+
|           Categoria|             Produto|               Marca| Preco|Avaliacao|Quant_Avaliacoes|Quant_Comentarios|                 URL|
+--------------------+--------------------+--------------------+------+---------+----------------+-----------------+--------------------+
|Tapete Higienico ...|Tapete Higiênico ...|               Petix| 74.49|      4.8|            2210|              373|https://www.merca...|
|   Racao Golden 15kg|Golden formula pr...|              Golden|191.28|      4.8|             600|              106|https://www.merca...|
|              Keldog|Bifinho para Cães...|              Keldog|  25.7|      4.9|             643|               96|https://www.merca...|
|Shampoo Cloresten...|1 Cloresten Shamp...|  Agener União Noxxi| 209.9|      4.9|             184|               46|https://www.merca...|
|               Camas|Cama Pet Cac

In [15]:
tratamento_nulos = tratamento_nulos.withColumn("Marca", when(col("Marca").isNull(), 'Nao Informado').otherwise(col("Marca")))

### Retirando campos repetidos

In [16]:
#Totais por coluna
tratamento_nulos.select([count(col(c)).alias(c) for c in tratamento_nulos.columns]).show()

+---------+-------+-----+-----+---------+----------------+-----------------+----+
|Categoria|Produto|Marca|Preco|Avaliacao|Quant_Avaliacoes|Quant_Comentarios| URL|
+---------+-------+-----+-----+---------+----------------+-----------------+----+
|     2280|   2280| 2280| 2266|     1954|            2280|             2280|2280|
+---------+-------+-----+-----+---------+----------------+-----------------+----+



In [17]:
#Retirando produtos repetidos
sem_repeticoes = tratamento_nulos.dropDuplicates()

In [18]:
#Totais por coluna
sem_repeticoes.select([count(col(c)).alias(c) for c in sem_repeticoes.columns]).show()

+---------+-------+-----+-----+---------+----------------+-----------------+----+
|Categoria|Produto|Marca|Preco|Avaliacao|Quant_Avaliacoes|Quant_Comentarios| URL|
+---------+-------+-----+-----+---------+----------------+-----------------+----+
|     2261|   2261| 2261| 2247|     1942|            2261|             2261|2261|
+---------+-------+-----+-----+---------+----------------+-----------------+----+



### Tratamento da variável "Categoria"

In [19]:
#Lista de categorias
categorias = sem_repeticoes.select("Categoria").distinct().collect()
lista =[row["Categoria"] for row in categorias]
lista_ordenada = sorted(lista)
lista_ordenada

['Afasta Gato',
 'Aminomix',
 'Anti-Latido',
 'Antipulgas Gatos',
 'Bandejas Sanitárias',
 'Banho A Seco Pet',
 'Banho Pet',
 'Bebedouros',
 'Bebedouros  e Comedouros',
 'Benebone',
 'Biscoito Cachorro',
 'Biscoitos para Cães',
 'Biscrok Pedigree',
 'Brinquedo Cachorro',
 'Brinquedo Interativo Cachorro',
 'Brinquedos Pet',
 'Cabana Pet',
 'Cachorro Forte',
 'Calcinhas',
 'Cama Cachorro',
 'Cama Cachorro Grande',
 'Cama Gato',
 'Cama Iglu',
 'Cama Nuvem',
 'Cama Pet',
 'Camas',
 'Caminha',
 'Casas',
 'Casinha Cachorro',
 'Casinha Gatos',
 'Cetodine Shampoo',
 'Cloresten',
 'Colchao Pet',
 'Comedouro',
 'Comedouro Cachorro',
 'Comedouro Gatos',
 'Comedouro Interativo',
 'Comedouros',
 'Dermogen Shampoo',
 'Doguitos',
 'Dolce Pet',
 'Episoothe Shampoo',
 'Escada Pet',
 'Frisbees',
 'Glicopan Pet',
 'Hexadene Shampoo',
 'Hidrapet',
 'Keldog',
 'Kit Cachorro',
 'Kit Pet',
 'Matacura',
 'Micodine Shampoo',
 'Organew',
 'Ossinho',
 'Osso Cachorro',
 'Ossos',
 'Pedigree Dentastix',
 'Perigot',

In [20]:
#Organização das categorias
categoria_tratadas = sem_repeticoes.withColumn("Categoria", when(col("Categoria").contains("Shampoo"), "Shampoo e Condicionadores").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Cloresten"), "Shampoo e Condicionadores").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Sebolytic"), "Shampoo e Condicionadores").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Sebotrat"), "Shampoo e Condicionadores").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Cama"), "Camas").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Caminha"), "Camas").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Plast Pet"), "Camas").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Cabana Pet"), "Camas").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Osso"), "Ossos").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Ossinho"), "Ossos").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Brinquedo"), "Brinquedos").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Benebone"), "Brinquedos").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Cachorro Forte"), "Brinquedos").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Kit"), "Kit").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Tapete"), "Tapete").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Zee Dog"), "Tapete").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Racao"), "Racao").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Rações"), "Racao").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Ração"), "Racao").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Special Dog"), "Racao").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Comedouro"), "Comedouro").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Biscoitos"), "Biscoitos").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Biscoito"), "Biscoitos").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Biscrok"), "Biscoitos").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Antipulgas"), "Antipulgas").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Petisco"), "Petisco").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Petiscao"), "Petisco").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Keldog"), "Petisco").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Doguitos"), "Petisco").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Pedigree Dentastix"), "Petisco").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Porta Gatos"), "Petisco").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Sebotrat"), "Sebotrat").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Casinhas"), "Casinhas").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Casas"), "Casinhas").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Casinha"), "Casinhas").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Toca Gatos"), "Casinhas").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Afasta Gato"), "Repelente").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Aminomix"), "Suplementos e Vitaminas").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Glicopan Pet"), "Suplementos e Vitaminas").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Pet Milk"), "Suplementos e Vitaminas").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Organew"), "Suplementos e Vitaminas").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Matacura"), "Suplementos e Vitaminas").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Pet Delicia"), "Sache").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Promun Dog"), "Sache").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Perigot"), "Banho").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Banho"), "Banho").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Dolce Pet"), "Banho").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Pet Smack"), "Banho").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Pet Clean"), "Banho").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Categoria").contains("Hidrapet"), "Banho").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Produto").contains("Brinquedo"), "Brinquedo").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Produto").contains("Kit"), "Kit").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Produto").contains("Bifinho"), "Petisco").otherwise(col("Categoria"))). \
withColumn("Categoria", when(col("Produto").contains("Finalizador"), "Banho").otherwise(col("Categoria")))

In [32]:
#Organização das Marcas
categoria_marca_tratadas = categoria_tratadas.withColumn("Marca", when(col("Marca").contains("inteligente"), 'Nao Informado').otherwise(col("Marca"))). \
withColumn("Marca", when(col("Marca").contains("CONTROLE"), 'Nao Informado').otherwise(col("Marca"))). \
withColumn("Marca", when(col("Marca").contains("Virbac"), "Virbac").otherwise(col("Marca"))). \
withColumn("Marca", when(col("Marca").contains("World"), "World Veterinaria").otherwise(col("Marca"))). \
withColumn("Marca", when(col("Marca").contains("Agener"), "Agener").otherwise(col("Marca"))). \
withColumn("Marca", when(col("Marca").contains("Society"), "Pet Society").otherwise(col("Marca"))). \
withColumn("Marca", when(col("Marca").contains("Perigot"), "Perigot").otherwise(col("Marca")))

In [33]:
categoria_marca_tratadas = categoria_marca_tratadas.withColumn("Marca", when(col("Marca").contains("AIC Matacura"), 'AIC MATACURA').otherwise(col("Marca")))

In [34]:
#Lista das Categorias após tratamento
categorias_tratadas = categoria_marca_tratadas.select("Categoria").distinct().collect()
lista = [row["Categoria"] for row in categorias_tratadas]
lista_ordenada = sorted(lista)
lista_ordenada
print(lista_ordenada, "\n", len(lista_ordenada))

['Antipulgas', 'Banho', 'Bebedouros', 'Biscoitos', 'Brinquedo', 'Camas', 'Casinhas', 'Colchao Pet', 'Comedouro', 'Escada Pet', 'Kit', 'Ossos', 'Petisco', 'Racao', 'Repelente', 'Sabonete', 'Sache', 'Shampoo e Condicionadores', 'Suplementos e Vitaminas', 'Tapete'] 
 20


### Tratamento da variável "Marcas"

In [35]:
#Verificação de Produtos com nome repetido
quantidade_produto = categoria_marca_tratadas.groupBy("Produto").agg(countDistinct("Categoria", "Marca").alias("Quant_Combinacoes"))
filtro_repetidos = quantidade_produto.filter(col("Quant_Combinacoes") > 1)
lista_produtos_repetidos = filtro_repetidos.select("Produto")
lista_produtos_repetidos = [row["Produto"] for row in filtro_repetidos.collect()]
df_sem_url = categoria_marca_tratadas.filter(col("Produto").isin(lista_produtos_repetidos))
df_sem_url.orderBy("Produto").show(50, truncate=False)
print(f"Existem {df_sem_url.count()} repetições")

+-------------------------+------------------------------------------------------------+-----------------------------------+------+---------+----------------+-----------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [36]:
#Verificação de duplicatas
df_sem_repeticoes = categoria_marca_tratadas.dropDuplicates()

In [37]:
categoria_marca_tratadas.count()

2261

In [27]:
df_sem_repeticoes.count()

2261

### Substituição da variável "URL" pela variável "ID" 

In [38]:
# Substituição da coluna "URL" pela coluna "ID"
id_urls = spark.read.csv(r"C:\Users\fred\Documents\CursoFullStack\Projeto\Web Scraping\Dados\IDs_URLs.csv", sep = ";", header = True)
df1_tratado = id_urls.join(df_sem_repeticoes, on="URL", how="inner")
df1_tratado = df1_tratado.drop("URL")
df1_tratado.show()

+----+--------------------+--------------------+--------------------+------+---------+----------------+-----------------+
|  ID|           Categoria|             Produto|               Marca| Preco|Avaliacao|Quant_Avaliacoes|Quant_Comentarios|
+----+--------------------+--------------------+--------------------+------+---------+----------------+-----------------+
|1709|               Camas|Colchonete Pet Ca...|            Binnopet| 179.9|      5.0|               7|                2|
|2041|               Camas|Cama Nuvem Pelúci...|             Tinenen|  36.0|      4.7|              19|                3|
|1389|               Racao|Pet Delícia Alime...|         Pet Delícia| 30.79|      4.9|             369|               49|
|1984|           Biscoitos|Snack Natural Des...|           PetTreats| 29.71|      4.1|              15|                3|
| 542|               Ossos|Osso Mordedor Com...|           Lumen Pet| 61.92|      4.7|              15|                2|
| 320|               Ban

In [40]:
# Criação do arquivo .parquet
df1_tratado.write.parquet(r"C:\Users\fred\Documents\CursoFullStack\Projeto\Web Scraping\Dados\df1_tratado.parquet")

In [41]:
df1_tratado.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Categoria: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Marca: string (nullable = true)
 |-- Preco: float (nullable = true)
 |-- Avaliacao: float (nullable = true)
 |-- Quant_Avaliacoes: integer (nullable = true)
 |-- Quant_Comentarios: integer (nullable = true)



In [39]:
df1_tratado.filter(col("Marca").isNull()).show()

+---+---------+-------+-----+-----+---------+----------------+-----------------+
| ID|Categoria|Produto|Marca|Preco|Avaliacao|Quant_Avaliacoes|Quant_Comentarios|
+---+---------+-------+-----+-----+---------+----------------+-----------------+
+---+---------+-------+-----+-----+---------+----------------+-----------------+

